In [76]:
import pandas as pd

In [77]:
# Carregando o DataFrame do arquivo CSV
df = pd.read_csv('/Users/anaxsouza/Documents/GitKraken/autologistics/input_ok.csv')

# Criando df_1 e removendo a coluna 'Soma de Total Unidades'
df_1 = df.drop(columns=['Soma de Total Unidades'])
# Ordenando df_1 pela coluna 'Demanda'
df_1 = df_1.sort_values(by=['Demanda'], ascending=False)

# Criando df_2 e removendo a coluna 'Demanda'
df_2 = df.drop(columns=['Demanda'])
# Ordenando df_2 pela coluna 'Soma de Total Unidades'
df_2 = df_2.sort_values(by=['Soma de Total Unidades'], ascending=False)

df_2

,CV,CM,Soma de Total Unidades
27,2814.0,50398676.0,17115.0
7,95839.0,50367554.0,12564.0
265,1925.0,50305782.0,8673.0
136,72147.0,50375500.0,7478.0
24,2829.0,50374194.0,7392.0
...,...,...,...
3085,100459.0,50337903.0,NaN
3089,121370.0,50399317.0,NaN
3092,136459.0,50438639.0,NaN
3094,135905.0,50437502.0,NaN


In [78]:
day_cycle = 31

dias_ciclo = int(day_cycle)
df_1['Demanda dia'] = df_1['Demanda'] / dias_ciclo
df_1['%_Previsão'] = df_1['Demanda dia'] / df_1['Demanda dia'].sum()

# Calculando a previsão acumulada
df_1['%ACC_Previsão'] = df_1['%_Previsão'].cumsum()

In [79]:
# Classificando conforme a previsão acumulada
def classificar_previsao(acc):
    if acc <= 0.602:
        return 'AA'
    elif acc <= 0.89:
        return 'A'
    elif acc <= 0.949:
        return 'B'
    elif acc <= 0.999:
        return 'C'
    else:
        return 'D'
    
df_1['ABC_Previsão'] = df_1['%ACC_Previsão'].apply(classificar_previsao)

In [80]:
backlog_days = 2.13

# Passo similar para backlog
dias_backlog = float(backlog_days)
df_2['PRD/DIA'] = df_2['Soma de Total Unidades'] / dias_backlog
df_2['%_BACKLOG'] = df_2['PRD/DIA'] / df_2['PRD/DIA'].sum()
df_2['%ACC_BACKLOG'] = df_2['%_BACKLOG'].cumsum()
df_2['ABC_BACKLOG'] = df_2['%ACC_BACKLOG'].apply(classificar_previsao)

In [81]:
# Ajuste final
pesos = {'AA': 9, 'A': 8, 'B': 7, 'C': 6, 'D': 5}

df = pd.merge(df_1, df_2, on='CM', how='left')

def ajuste_final(previsao, backlog):
    if pesos[previsao] > pesos[backlog]:
        return previsao
    else:
        return backlog

df['ABC_AJUSTADO'] = df.apply(lambda x: ajuste_final(x['ABC_Previsão'], x['ABC_BACKLOG']), axis=1)

# Exibir resultado final
print(df)

df.to_csv('/Users/anaxsouza/Documents/GitKraken/autologistics/input_final.csv', index=False)  # Salva o DataFrame em um arquivo CSV

          CV_x          CM        Demanda  Demanda dia    %_Previsão  \
0      95840.0  50369418.0  271520.379644  8758.721924  2.034638e-02   
1      96703.0  50315466.0  232947.081081  7514.421970  1.745589e-02   
2     148288.0  50463576.0  151049.180960  4872.554225  1.131887e-02   
3     135733.0  50437335.0  149785.802650  4831.800085  1.122420e-02   
4     135761.0  50437365.0  137840.015065  4446.452099  1.032904e-02   
...        ...         ...            ...          ...           ...   
3092  136459.0  50438639.0       2.162162     0.069747  1.620217e-07   
3093  135365.0  50436651.0       1.729730     0.055798  1.296173e-07   
3094  135905.0  50437502.0       1.297297     0.041848  9.721300e-08   
3095  135939.0  50439808.0       1.072306     0.034591  8.035329e-08   
3096  110421.0  50365741.0       1.000000     0.032258  7.493502e-08   

      %ACC_Previsão ABC_Previsão      CV_y  Soma de Total Unidades  \
0          0.020346           AA   95840.0                    46.